Maryland Department of Planning
Property Map Products
https://planning.maryland.gov/Pages/OurProducts/PropertyMapProducts/FinderProduct.aspx
>> Open Data GIS Downloads
MdProperty View
Dorchester
https://planning.maryland.gov/Pages/OurProducts/DownloadFiles.aspx

In [ ]:
!pip install leven
!pip install simpledbf
!pip install ngram

In [ ]:
import pandas as pd
from simpledbf import Dbf5
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Latest SDAT, updated to current 6-month pull

In [ ]:
dbf = Dbf5('drive/My Drive/pita 2020/Dorc2017.dbf')
df = dbf.to_dataframe()
dbf2 = Dbf5('drive/My Drive/pita 2020/Dorc0620.dbf')
df2 = dbf2.to_dataframe()
updates = df2.acctid.array
df = df.query('acctid not in @updates').append(df2)
df.acctid = df.acctid.apply(lambda x: str(x).strip())

In [ ]:
dbf2 = Dbf5('drive/My Drive/pita 2020/Dorc0620.dbf')

In [ ]:
df.query('acctid == "1007147813"').to_dict()
#  print(k,v)

{'acctid': {603: '1007147813'},
 'acres': {603: 0.304},
 'address': {603: '311 BELVEDERE AVE'},
 'addrtyp': {603: nan},
 'agfndarea': {603: nan},
 'agfndluom': {603: nan},
 'aprtment': {603: nan},
 'assessor': {603: '1018'},
 'bg2010': {603: '240199704004'},
 'bldg_story': {603: nan},
 'bldg_units': {603: nan},
 'block': {603: nan},
 'city': {603: 'CAMBRIDGE'},
 'ciuse': {603: nan},
 'considr1': {603: nan},
 'convey1': {603: '4'},
 'crtarcod': {603: nan},
 'ct2010': {603: '24019970400'},
 'curimpvl': {603: 102200.0},
 'curlndvl': {603: 41900.0},
 'curttlvl': {603: nan},
 'depth': {603: nan},
 'descbldg': {603: 'DWEL Standard Unit'},
 'descciuse': {603: nan},
 'desccnst': {603: 'CNST Siding'},
 'descexcl': {603: 'N/A Taxable Property'},
 'descgrad': {603: 'Codes range from lowest to highest quality 1-9'},
 'desclu': {603: 'Residential'},
 'descstyl': {603: 'STRY 1 1/2 Story No Basement'},
 'descsubd': {603: 'N/A'},
 'desctown': {603: 'Cambridge'},
 'digxcord': {603: 479875.6},
 'digycor

#### Rental license information, by taxid

In [ ]:
# this is a stand-in while waiting for the latest dataset
rentalinfo_columns = ['ACCTID','Effective Date', 'Expiration Date', 'State Id', 'Status','Property Location', 'Dist/Account No', 'RENTAL', '_c12','LICENSE_TYPE', 'UNITS']
license_df = pd.read_csv("drive/My Drive/pita 2020/20191225-can_rental_study_updated.csv")#[rentalinfo_columns]
license_df.rename(columns={'ACCTID':'acctid'},inplace=True)
license_df.acctid = license_df.acctid.apply(lambda x: str(x).strip())
license_df = license_df.query('LICENSE_TYPE =="RENTAL"')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,9,10,28,43,85,109,110) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# rentals = df.merge(license_df,on='acctid',how='outer',indicator=True)
# rentals.query('_merge == "right_only"')

## NOTE, THIS DROPS THE WOODS ROAD BUILDINGS !!!!
from collections import defaultdict 

licensed_accts = set(license_df.acctid.array)
rentals = df.merge(license_df,on='acctid',how='inner')#,indicator=True)
rental_accts = set(rentals.acctid.array)
split_accts = licensed_accts.difference(rental_accts)
split_accts.add('1007127049-1')
split_accts = {(acct.split("-")[0],acct.split("-")[1]) for acct in split_accts }
acct_splits = defaultdict(list)
for k,v in split_accts:
  acct_splits[k].append(v)

# make a df with the expanded reccords in it
def add_extras(acct_splits,split=False):
  df_extras = pd.DataFrame(columns=df.columns)
  for acct in acct_splits.keys():
    row = df.query('acctid == @acct').copy()
    if split:
      for variant in acct_splits[acct]:
        row.acctid = acct+"-"+str(variant)
    df_extras = df_extras.append(row)
  return df_extras

rentals = rentals.append(add_extras(acct_splits).merge(license_df,on='acctid',how='inner'))

In [ ]:
rentals.reset_index(drop=True)

,jurscode,acctid,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,...,RZREALDAT,SDATWEBADR,SECTION,SEQNUMB,SPECIAL,SQFTSTRC,STRTDIR,STRTNAM,STRTNUM,STRTSFX,STRTTYP,STRTUNT,STRUBLDG,STRUCNST,STRUGRAD,STRUSTYL,SUBDIVSN,TOWNCODE,TRADATE,TRAILER,TRANSNO1,WIDTH,YEARBLT,ZIPCODE,ZNCHGDAT,ZONING,License Id,License Type Id,Business Name,Customer Id,Effective Date,Expiration Date,State Id,Status,Property Location,Dist/Account No,RENTAL,_c12,LICENSE_TYPE,UNITS
0,DORC,1007104057,481347.2,100263.6,24019970400,240199704002,81,N,SF,304 DORCHESTER AVE,304,NaN,DORCHESTER,AVE,NaN,NaN,P,CAMBRIDGE,21613,FITZHUGH WILLIAM R &,FITZHUGH WILLIAM R JR,FITZHUGH WILLIAM R &,5522 WHITEHALL RD,NaN,CAMBRIDGE,MD,21613,3442,304,NaN,DORCHESTER,AVE,CAMBRIDGE,21613,NaN,IMPSLT 22 AND P/O #21,"14,700 SQ. FT.",304 DORCHESTER AVE,PLC,00300,...,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,NaN,3635,0,1368,NaN,DORCHESTER,304.0,NaN,AVE,NaN,1,1,4.0,3,NaN,3.0,19940301.0,0,940710.0,0.0,1945.0,21613.0,NaN,R-3,19-00045,RENTAL,WILLIAM R FITZHUGH SR,RR-00416,7/1/2019,6/30/2020,NaN,Approved,304 DORCHESTER AVE DOWN 437,07-104057,2.0,NaN,RENTAL,2.0
1,DORC,1007104073,480302.8,99425.4,24019970600,240199706001,81,N,SF,415 SHEPHERD AVE,415,NaN,SHEPHERD,AVE,NaN,NaN,P,CAMBRIDGE,21613,RUDDY DUCK LLC,NaN,RUDDY DUCK LLC,PO BOX 1181,NaN,CAMBRIDGE,MD,21613,NaN,415,NaN,SHEPHERD,AVE,CAMBRIDGE,21613,NaN,"IMPSLOT 84&85-15,696 SQ.",415 SHEPHERD AVENUE,CAMBRIDGE,NaN,01335,...,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,NaN,12772,0,2664,NaN,SHEPHERD,415.0,NaN,AVE,NaN,1,1,4.0,5,NaN,3.0,20160615.0,0,151278.0,0.0,1971.0,21613.0,NaN,R-2,19-00571,RENTAL,RUDDY DUCK LLC,RR-05536,7/1/2019,6/30/2020,NaN,Approved,415 SHEPHERD AVE 5078,07-104073,2.0,NaN,RENTAL,2.0
2,DORC,1007104294,480556.3,99908.5,24019970600,240199706001,81,N,SF,727 PEACHBLOSSOM AVE,727,NaN,PEACHBLOSSOM,AVE,NaN,NaN,P,CAMBRIDGE,21613,RUDDY DUCK LLC,A MARYLAND LIMITED LIABILITY COMPA,RUDDY DUCK LLC,PO BOX 1181,NaN,CAMBRIDGE,MD,21613,NaN,727,NaN,PEACHBLOSSOM,AVE,CAMBRIDGE,21613,NaN,"IMPSLOT 3,700 SQ. FT.",727 PEACHBLOSSOM AVENUE,CAMBRIDGE,NaN,01255,...,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,NaN,10382,0,1132,NaN,PEACHBLOSSOM,727.0,NaN,AVE,NaN,1,1,4.0,1,NaN,3.0,20150316.0,0,140835.0,0.0,1930.0,21613.0,NaN,R-3,19-00006,RENTAL,RUDDY DUCK LLC,RR-00380,7/1/2019,6/30/2020,NaN,Approved,727 PEACHBLOSSOM AVE 1375,07-104294,1.0,NaN,RENTAL,1.0
3,DORC,1007104324,480198.9,99773.7,24019970600,240199706002,81,N,SF,900 CENTRAL AVE,900,NaN,CENTRAL,AVE,NaN,NaN,P,CAMBRIDGE,21613,SHARP MARTIN,NaN,SHARP MARTIN,3723 SWANN RD,NaN,SUITLAND,MD,20746,2235,900,NaN,CENTRAL,AVE,CAMBRIDGE,21613,NaN,"IMPSLOT 3,979 SQ. FT.",900 CENTRAL AVENUE,CAMBRIDGE,MLB,00582,...,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,NaN,2510,0,930,NaN,CENTRAL,900.0,NaN,AVE,NaN,1,4,3.0,1,NaN,3.0,20040617.0,0,31681.0,0.0,1940.0,21613.0,NaN,C-2,19-00498,RENTAL,MARTIN SHARP,RR-03716,7/1/2019,6/30/2020,NaN,Approved,900 CENTRAL AVE 3031,07-104324,1.0,NaN,RENTAL,1.0
4,DORC,1007104340,480195.8,99750.0,24019970600,240199706002,81,N,SF,904 CENTRAL AVE,904,NaN,CENTRAL,AVE,NaN,NaN,P,CAMBRIDGE,21613,NEWCOMB JAMES C JR AND ETAL,NEWCOMB JOYCE A,NEWCOMB JAMES C JR AND ET,906 CENTRAL AVENUE,NaN,CAMBRIDGE,MD,21613,2724,904,NaN,CENTRAL,AVE,CAMBRIDGE,21613,NaN,"IMPSLOT 4,000 SQ. FT.",904 CENTRAL AVENUE,CAMBRIDGE,NaN,01084,...,NaN,http://sdat.dat.maryland.gov/RealProperty/Page...,NaN,2514,0,792,NaN,CENTRAL,904.0,NaN,AVE,NaN,1,1,4.0,1,NaN,3.0,20120508.0,0,110821.0,0.0,1940.0,21613.0,NaN,C-2,19-00497,RENTAL,JAMES NEWCOMB JR,RR-04380,7/1/2019,6/30/2020,NaN,Approved,904 CENTRAL AVE 3029,07-104340,1.0,NaN,RENTAL,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
rentals.rename(columns={'Business Name':'Business_Name'}).fillna("").query('ownname1.str.contains("CEPHA")',engine='python')

,jurscode,acctid,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,...,RZREALDAT,SDATWEBADR,SECTION,SEQNUMB,SPECIAL,SQFTSTRC,STRTDIR,STRTNAM,STRTNUM,STRTSFX,STRTTYP,STRTUNT,STRUBLDG,STRUCNST,STRUGRAD,STRUSTYL,SUBDIVSN,TOWNCODE,TRADATE,TRAILER,TRANSNO1,WIDTH,YEARBLT,ZIPCODE,ZNCHGDAT,ZONING,License Id,License Type Id,Business_Name,Customer Id,Effective Date,Expiration Date,State Id,Status,Property Location,Dist/Account No,RENTAL,_c12,LICENSE_TYPE,UNITS
268,DORC,1007124236,479954.1,100109.3,24019970500,240199705001,81,N,SF,627 DOUGLAS ST,627,,DOUGLAS,ST,,,P,CAMBRIDGE,21613,CEPHAS GILBERT C,,CEPHAS GILBERT C,P.O. BOX 832,,CAMBRIDGE,MD,21613,,627,,DOUGLAS,ST,CAMBRIDGE,21613,,"IMPSLOT 4,050 SQ. FT.",627 DOUGLAS STREET,CAMBRIDGE,PLC,00285,...,,http://sdat.dat.maryland.gov/RealProperty/Page...,,3692,0,1148,,DOUGLAS,627,,ST,,1,1,3,1,,3.0,1.99303e+07,0,930753,0.0,1935,21613,,R-4,19-00365,RENTAL,GILBERT CEPHAS,RR-03642,7/1/2019,6/30/2020,,Approved,627 DOUGLAS ST 3096,07-124236,1.0,,RENTAL,1.0
403,DORC,1007131682,479716.2,100091.3,24019970500,240199705001,81,N,SF,743 WASHINGTON ST,743,,WASHINGTON,ST,,,P,CAMBRIDGE,21613,CEPHAS ROSE & TROY,,CEPHAS ROSE & TROY,830 FAIRMOUNT AVE,,CAMBRIDGE,MD,21613,2151,743,,WASHINGTON,ST,CAMBRIDGE,21613,,"IMPSLOT 23-2,000 SQ. FT.",743 WASHINGTON STREET,CAMBRIDGE,MLB,00344,...,,http://sdat.dat.maryland.gov/RealProperty/Page...,,14651,0,1008,,WASHINGTON,743,,ST,,1,1,3,5,,3.0,1.99609e+07,0,970250,0.0,1920,21613,,R-4,19-00830,RENTAL,TROY CEPHAS,RR-07323,7/1/2019,6/30/2020,,Approved,743 WASHINGTON ST 2121,07-131682,1.0,,RENTAL,1.0
770,DORC,1007152051,480076.4,99921.0,24019970600,240199706003,81,N,SF,809 PINE ST,809,,PINE,ST,,,P,CAMBRIDGE,21613,CEPHAS JAMES E AND MATTHEWS,YOLANDA S,CEPHAS JAMES E AND MATTHE,405 J. BROOKRIDGE DR,,SALISBURY,MD,21804,,809,,PINE,ST,CAMBRIDGE,21613,,"IMPSLOT 3,100 SQ. FT.",809 PINE STREET,CAMBRIDGE,MLB,00783,...,,http://sdat.dat.maryland.gov/RealProperty/Page...,,10751,0,762,,PINE,809,,ST,,1,1,3,5,,3.0,2.00701e+07,0,60879,0.0,1930,21613,,R-3,19-00638,RENTAL,JAMES CEPHAS,RR-04400,7/1/2019,6/30/2020,,Approved,809 PINE ST 1527,07-152051,1.0,,RENTAL,1.0
944,DORC,1007164467,480222.5,99808.8,24019970600,240199706002,81,N,SF,502 ROBBINS ST,502,,ROBBINS,ST,,,P,CAMBRIDGE,21613,CEPHAS JAMES EDWARD,WELCH YOLANDA SHAVON,CEPHAS JAMES EDWARD,405 J BROOKRIDGE DR,,SALISBURY,MD,21804,,502,,ROBBINS,ST,CAMBRIDGE,21613,,"IMPSLOT 6,000 SQ. FT.",502 ROBBINS STREET,CAMBRIDGE,,01345,...,,http://sdat.dat.maryland.gov/RealProperty/Page...,,12083,0,1804,,ROBBINS,502,,ST,,1,1,3,5,,3.0,2.01608e+07,0,160106,0.0,1925,21613,,C-2,19-00584,RENTAL,JAMES CEPHAS,RR-04400,7/1/2019,6/30/2020,,Approved,502 ROBBINS ST 1900,07-164467,2.0,,RENTAL,2.0
1102,DORC,1007172168,480114.5,99872.7,24019970600,240199706002,81,N,SF,822 PINE ST,822,,PINE,ST,,,P,CAMBRIDGE,21613,CEPHAS JAMES E AND MATTHEWS,YOLANDA,CEPHAS JAMES E AND MATTHE,405 J. BROOKRIDGE DR,,SALISBURY,MD,21804,,822,,PINE,ST,CAMBRIDGE,21613,,"IMPSLOT 2,700 SQ. FT.",822 PINE STREET,CAMBRIDGE,MLB,00783,...,,http://sdat.dat.maryland.gov/RealProperty/Page...,,10764,0,1120,,PINE,822,,ST,,1,1,3,5,,3.0,2.00701e+07,0,60877,0.0,1930,21613,,C-2,19-00502,RENTAL,JAMES CEPHAS,RR-04400,7/1/2019,6/30/2020,,Approved,822 PINE ST 1541,07-172168,1.0,,RENTAL,1.0


In [ ]:
!cp "drive/My Drive/pita 2020/text_cluster.py" .
from text_cluster import assign_clusters
from text_cluster import text_similarity
from text_cluster import cluster_strings


####Cluster the rental licenses
Assign clusters by owner name and owner address

In [ ]:
%%time
owner_names = cluster_strings(rentals.ownname1.array,'o_name',metric='jaccard',threshold=0.1)
owner_addresses = cluster_strings(rentals.ownadd1.astype(str).array,'o_address',metric='jaccard',threshold=0.1)

rows: 1547 clusters: 173
rows: 1547 clusters: 163
CPU times: user 3min 48s, sys: 1.75 s, total: 3min 49s
Wall time: 3min 47s


In [ ]:
def best_cluster(cnumbers):
  if len(cnumbers) > 1 and cnumbers.index[0] < 0:
    return cnumbers.index[1]
  else:
    return cnumbers.index[0]

rentals['owner_cluster'] = rentals.apply(lambda x: owner_names.get(x.ownname1,-1), axis=1)
rentals['owneradd_cluster'] = rentals.apply(lambda x: owner_addresses.get(x.ownadd1,-1), axis=1)
rentals[['acctid','ownname1','owner_cluster','ownadd1','owneradd_cluster']]

    
ro_df = pd.DataFrame(rentals.groupby(['acctid'])['owneradd_cluster'].agg(lambda x:best_cluster(x.value_counts())))#.to_dict()
roadd_c = rentals.groupby(['acctid'])['owner_cluster'].agg(lambda x:best_cluster(x.value_counts()))
ro_df = ro_df.merge(roadd_c,left_index=True,right_index=True)
ro_df

,owneradd_cluster,owner_cluster
acctid,,
1007104057,164,174
1007104073,0,0
1007104294,0,0
1007104324,1,1
1007104340,165,175
...,...,...
1007239653,619,693
1007286609,146,152
1007286611,146,163


#### Save rental clusters

In [ ]:
ro_df.to_csv('drive/My Drive/pita 2020/rental_clusters.csv')